In [25]:
!pip uninstall scikit-learn scikit-survival
#!pip show scikit-learn
!pip install scikit-learn scikit-survival lifelines
#!pip install scikit-learn==1.1.0
#!pip install scikit-survival==0.18.0
#! pip install eli5==0.13.0

Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/scikit_learn-1.5.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.11/dist-packages/sklearn/*
Proceed (Y/n)? Y
  Successfully uninstalled scikit-learn-1.5.2
Found existing installation: scikit-survival 0.23.1
Uninstalling scikit-survival-0.23.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/scikit_survival-0.23.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/sksurv/*
Proceed (Y/n)? Y
  Successfully uninstalled scikit-survival-0.23.1
Y
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached scikit_survival-0.23.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (48 kB)
  Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.

In [26]:
### import necessary packages

import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import scipy.stats as stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from itertools import product
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn import tree
import pydotplus
import os

from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest

#from eli5.sklearn import PermutationImportance
from sklearn.metrics import roc_curve, roc_auc_score, f1_score

from sklearn.utils import check_random_state
from sklearn.model_selection import check_cv
from sklearn.utils.metaestimators import available_if
from sklearn.utils import check_array, check_random_state
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# Lire le fichier en sautant la première ligne
df = pd.read_excel("/content/drive/MyDrive/GastricCancerData.xlsx", skiprows=1)
# Afficher les premières lignes du dataframe
df.head()

,AGE,SEXE,Cardiopathie,Ulceregastrique,Douleurepigastrique,Ulcero-bourgeonnant,Constipation,Denitrution,Tabac,Mucineux,Tubuleux,Infiltrant,Stenosant,Metastases,Adenopathie,Traitement,Tempsdesuivi (Mois),Deces
0,65,F,NON,NON,OUI,NON,OUI,OUI,OUI,OUI,NON,OUI,OUI,OUI,OUI,Chirurgie_Exclusive,1,OUI
1,34,F,NON,NON,OUI,NON,OUI,NON,OUI,OUI,OUI,OUI,OUI,OUI,OUI,Chirurgie_Chimiotherapie,1,OUI
2,55,H,NON,NON,OUI,NON,OUI,NON,OUI,OUI,NON,OUI,NON,OUI,OUI,Chirurgie_Exclusive,1,OUI
3,60,F,OUI,OUI,OUI,NON,OUI,NON,OUI,OUI,OUI,OUI,NON,OUI,NON,Chirurgie_Exclusive,2,NON
4,65,H,OUI,NON,OUI,NON,OUI,NON,OUI,NON,NON,OUI,OUI,NON,OUI,Chirurgie_Chimiotherapie,2,OUI


In [29]:
#df = pd.read_excel("D:/UADB/MEMOIRE/Data/GastricCancerData.xlsx")

In [30]:
#affichage des dimensions
df.shape

(337, 18)

In [31]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Identifier les colonnes catégoriques
CatCols = df.select_dtypes(include=['object']).columns
# Encodage des variables catégoriques
#Label Encoding
label_encoder = LabelEncoder()
for col in CatCols:
    df[col] = label_encoder.fit_transform(df[col].astype(str))

df.head()

,AGE,SEXE,Cardiopathie,Ulceregastrique,Douleurepigastrique,Ulcero-bourgeonnant,Constipation,Denitrution,Tabac,Mucineux,Tubuleux,Infiltrant,Stenosant,Metastases,Adenopathie,Traitement,Tempsdesuivi (Mois),Deces
0,65,0,0,0,1,0,1,1,1,1,0,1,1,1,1,1,1,1
1,34,0,0,0,1,0,1,0,1,1,1,1,1,1,1,0,1,1
2,55,1,0,0,1,0,1,0,1,1,0,1,0,1,1,1,1,1
3,60,0,1,1,1,0,1,0,1,1,1,1,0,1,0,1,2,0
4,65,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,2,1


In [32]:
# Effacer les variables non pertinantes
clean=['SEXE','Tubuleux','Traitement','Constipation']
df=df.drop(columns=clean,axis=1)


## 3. **Deep Survival Models**


**Données employées**  
Les variables cliniques et tumorales retenues pour l'analyse du risque de décès chez les patients atteints d'un cancer de l'estomac ont été sélectionnées selon leur pertinence statistique dans le modèle de Cox et leur contribution explicative mesurée par les valeurs SHAP. Parmi ces variables figuraient :**l'âge des patients**, la présence de **comorbidités (cardiopathies)**, des **symptômes cliniques** (douleur épigastrique, ulcère gastrique, lésion ulcéro-bourgeonnante), des **facteurs liés au mode de vie** (tabagisme, dénutrition), ainsi que des **caractéristiques tumorales** spécifiques (forme mucineuse, infiltration, sténose, métastases et adénopathies). Ces critères ont été retenus car ils ont systématiquement démontré, à travers le modèle de Cox et leurs valeurs SHAP positives, une influence significative sur la réduction de la survie. La robustesse de ces variables, validée à la fois par une approche statistique classique et par une analyse d'interprétabilité moderne (SHAP), fournit une base solide pour les explorations ultérieures.
Dans ce chapitres, nous étendrons cette analyse à des approches de *machine learning*, afin de comparer les performances prédictives et les insights cliniques entre méthodes traditionnelles et algorithmes avancés.  

In [34]:
n_censored = df.shape[0] - df["Deces"].sum()
print(f"{n_censored / df.shape[0] * 100:.1f}% des enregistrements sont censurés")

33.8% des enregistrements sont censurés


## **IV. Deep Survival Models**

In [ ]:
!pip install pycox lifelines DeepSurv

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from lifelines.statistics import (logrank_test, pairwise_logrank_test, multivariate_logrank_test, survival_difference_at_fixed_point_in_time_test)

In [ ]:
# Lire le fichier en sautant la première ligne
df = pd.read_excel("/content/drive/MyDrive/GastricCancerData.xlsx", skiprows=1)
# Afficher les premières lignes du dataframe

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Identifier les colonnes catégoriques
CatCols = df.select_dtypes(include=['object']).columns
# Encodage des variables catégoriques
#Label Encoding
label_encoder = LabelEncoder()
for col in CatCols:
    df[col] = label_encoder.fit_transform(df[col].astype(str))
df.head()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input, Dense, Dropout

vars = ["AGE",'SEXE', 'Cardiopathie', 'Ulceregastrique', 'Douleurepigastrique',
        'Ulcero-bourgeonnant', 'Constipation', 'Denitrution', 'Tabac',
        'Mucineux', 'Tubuleux', 'Infiltrant', 'Stenosant', 'Metastases',
        'Adenopathie', 'Traitement']
# Séparer covariables, temps et statut
X = df[vars].values
T = df['Tempsdesuivi (Mois)'].values
E = df['Deces'].values

# Définir le modèle DeepSurv
def build_deepsurv_model(input_dim):
    model = Sequential([
        Dense(512, activation='relu', input_dim=input_dim),
        Dense(64, activation='relu'),
        Dense(1, activation=None)  # Pas d'activation pour la couche finale
    ])
    return model

# Créer le modèle
input_dim = X.shape[1]  # Nombre de covariables
model = build_deepsurv_model(input_dim)


In [ ]:
#Implémentation manuelle de la perte de Cox
import tensorflow.keras.backend as K

def cox_ph_loss(y_true, y_pred):
    # y_true[:, 0]: temps d'événement ou censure
    # y_true[:, 1]: statut (1 = événement, 0 = censure)
    events = tf.cast(y_true[:, 1], dtype=tf.bool)
    risk_set = tf.math.cumsum(tf.exp(y_pred), reverse=True)
    log_risk = tf.math.log(risk_set)
    loss = tf.reduce_sum(y_pred[events] - log_risk[events])
    return -loss

In [ ]:
# Compiler le modèle
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=cox_ph_loss)

# Préparer les données pour l'entraînement
y_train = np.vstack((T, E)).T  # Combiner temps et statut
history = model.fit(X, y_train, batch_size=32, epochs=100, verbose=1)


In [ ]:
#Évaluer le modèle
from lifelines.utils import concordance_index

# Prédire les scores de risque
risk_scores = model.predict(X).flatten()

# Calculer le C-index
c_index = concordance_index(T, -risk_scores, E)
print(f"C-index: {c_index}")

In [ ]:
# Log-vraisemblance
def log_likelihood(y_true, y_pred):
    # Extraire le temps et l'événement (censure)
    T, E = y_true[:, 0], y_true[:, 1]

    # Calcul du risque cumulatif basé sur les prédictions
    risk_set = np.cumsum(np.exp(y_pred), axis=0)[::-1]
    log_risk = np.log(risk_set)

    # Calcul de la log-vraisemblance
    log_likelihood = np.sum(E * (y_pred - log_risk))
    return log_likelihood

#Score du Brier
from sklearn.metrics import mean_squared_error

def brier_score(y_true, y_pred):
    # Calculer les probabilités de survie
    survival_prob = np.exp(-np.exp(y_pred))

    # Calculer le Brier Score
    score = mean_squared_error(y_true[:, 1], survival_prob)
    return score

# Indice C (indice de concordance)
from lifelines.utils import concordance_index

def c_index(y_true, y_pred):
    # Le concordance_index prend en entrée le temps et les événements
    return concordance_index(y_true[:, 0], -y_pred[:, 0], event_observed=y_true[:, 1])


# Prédictions du modèle
y_pred = model.predict(X)

# Calcul des métriques
brier_score_value = brier_score(y_train, y_pred)
c_index_value = c_index(y_train, y_pred)

print(f"Brier Score: {brier_score_value}")
print(f"C-Index: {c_index_value}")


In [ ]:
!pip install keras_tuner

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import keras_tuner as kt
import numpy as np
from lifelines.utils import concordance_index

# Fonction pour construire le modèle DeepSurv
def build_deepsurv_model(hp):
    model = Sequential()
    # Ajout de couches avec optimisation des unités
    model.add(Dense(units=hp.Int('units_layer_1', min_value=64, max_value=512, step=64),
                    activation=hp.Choice('activation_layer_1', values=['relu', 'tanh']),
                    input_dim=X.shape[1]))
    model.add(Dense(units=hp.Int('units_layer_2', min_value=32, max_value=128, step=32),
                    activation=hp.Choice('activation_layer_2', values=['relu', 'tanh'])))
    model.add(Dense(1, activation=None))  # Pas d'activation pour la couche finale

    # Optimiseur avec apprentissage optimisé
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss=cox_ph_loss
    )
    return model

# Fonction personnalisée pour la perte Cox PH
def cox_ph_loss(y_true, y_pred):
    events = tf.cast(y_true[:, 1], dtype=tf.bool)
    risk_set = tf.math.cumsum(tf.exp(y_pred), reverse=True)
    log_risk = tf.math.log(risk_set)
    loss = tf.reduce_sum(y_pred[events] - log_risk[events])
    return -loss

# Préparer les données
y_train = np.vstack((T, E)).T  # Combiner temps et statut

# Recherche des meilleurs hyperparamètres
tuner = kt.Hyperband(
    build_deepsurv_model,
    objective=kt.Objective('val_loss', direction='min'),  # Minimiser la perte
    max_epochs=20,
    factor=3,
    directory='hyperparameter_tuning',
    project_name='deepsurv_optimization'
)

# Définir les données de validation
tuner.search(X, y_train, validation_split=0.2, epochs=20, batch_size=32, verbose=1)

# Récupérer les meilleurs hyperparamètres
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Meilleurs hyperparamètres : {best_hps.values}")

# Construire le meilleur modèle
best_model = tuner.hypermodel.build(best_hps)

# Entraîner le meilleur modèle
history = best_model.fit(X, y_train, batch_size=32, epochs=50, validation_split=0.2)

# Calculer le C-index
risk_scores = best_model.predict(X).flatten()
c_index = concordance_index(T, -risk_scores, E)
print(f"C-index du meilleur modèle : {c_index}")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from lifelines.utils import concordance_index

# Définir le modèle DeepSurv
def build_deepsurv_model(input_dim):
    model = Sequential([
        Dense(512, activation='relu', input_dim=input_dim),
        Dense(64, activation='relu'),
        Dense(1, activation=None)  # Pas d'activation pour la couche finale
    ])
    return model

# Préparer les données (X : covariables, T : temps, E : statut)
# Exemple : X, T, E doivent être fournis
# X : matrice des covariables
# T : temps d'événement ou de censure
# E : 1 si événement, 0 si censure

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, T_train, T_test, E_train, E_test = train_test_split(
    X, T, E, test_size=0.2, random_state=42
)

# Créer le modèle
input_dim = X_train.shape[1]  # Nombre de covariables
model = build_deepsurv_model(input_dim)

# Définir la fonction de perte Cox PH
def cox_ph_loss(y_true, y_pred):
    # y_true[:, 0]: temps d'événement ou censure
    # y_true[:, 1]: statut (1 = événement, 0 = censure)
    events = tf.cast(y_true[:, 1], dtype=tf.bool)
    risk_set = tf.math.cumsum(tf.exp(y_pred), reverse=True)
    log_risk = tf.math.log(risk_set)
    loss = tf.reduce_sum(y_pred[events] - log_risk[events])
    return -loss

# Compiler le modèle
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=cox_ph_loss)

# Préparer les données pour l'entraînement
y_train = np.vstack((T_train, E_train)).T  # Combiner temps et statut pour le train
y_test = np.vstack((T_test, E_test)).T  # Combiner temps et statut pour le test

# Entraîner le modèle
history = model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1)

# Évaluer sur les données de test
risk_scores_test = model.predict(X_test).flatten()
c_index_test = concordance_index(T_test, -risk_scores_test, E_test)
print(f"C-index sur les données de test : {c_index_test}")
